In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Note: TensorFlow is not needed for PyNetwork to work. It's only used to load the dataset
import tensorflow as tf

import PyNetwork

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

shape = (28, 28)
x_train = x_train.reshape(-1, *shape) / 255
x_test = x_test.reshape(-1, *shape) / 255

labels = np.eye(10)

y_train = labels[y_train.flatten()]
y_test = labels[y_test.flatten()]

# Large Learning Rate

In [5]:
model = PyNetwork.Sequential()

model.add( PyNetwork.layers.Input((28, 28)) )
model.add( PyNetwork.layers.Flatten() )
model.add( PyNetwork.layers.Dense(100, activation_function='relu', l2=0.01, l1=0.0) )
model.add( PyNetwork.layers.BatchNorm() )
model.add( PyNetwork.layers.Dense(10, activation_function='softmax', l2=0.0, l1=0.0) )

optimizer = PyNetwork.optimizers.RMSprop(learning_rate=0.0005)
model.build(loss_function='cross_entropy', optimizer=optimizer, metrics='accuracy')

In [6]:
%%time
model.train(x_train, y_train, epochs=10, batch_size=128, verbose=True)

Training on 60000 samples
Epoch 1/10
cross_entropy: 1.9130 - accuracy: 0.3438
Training on 60000 samples
Epoch 2/10
cross_entropy: 2.0788 - accuracy: 0.2344
Training on 60000 samples
Epoch 3/10
cross_entropy: 1.8481 - accuracy: 0.3672
Training on 60000 samples
Epoch 4/10
cross_entropy: 1.9526 - accuracy: 0.2500
Training on 60000 samples
Epoch 5/10
cross_entropy: 1.7199 - accuracy: 0.4062
Training on 60000 samples
Epoch 6/10
cross_entropy: 2.1880 - accuracy: 0.1172
Training on 60000 samples
Epoch 7/10
cross_entropy: 1.9158 - accuracy: 0.2031
Training on 60000 samples
Epoch 8/10
cross_entropy: 2.0658 - accuracy: 0.1641
Training on 60000 samples
Epoch 9/10
cross_entropy: 2.1944 - accuracy: 0.1172
Training on 60000 samples
Epoch 10/10
cross_entropy: 2.2708 - accuracy: 0.1328
CPU times: user 3min 17s, sys: 12.9 s, total: 3min 30s
Wall time: 23.7 s


In [7]:
model.evaluate(x_test, y_test)

'cross_entropy: 2.2318 - accuracy: 0.1356'

# Smaller Learning Rate

In [9]:
model = PyNetwork.Sequential()

model.add( PyNetwork.layers.Input((28, 28)) )
model.add( PyNetwork.layers.Flatten() )
model.add( PyNetwork.layers.Dense(100, activation_function='relu', l2=0.01, l1=0.0) )
model.add( PyNetwork.layers.BatchNorm() )
model.add( PyNetwork.layers.Dense(10, activation_function='softmax', l2=0.0, l1=0.0) )

optimizer = PyNetwork.optimizers.RMSprop(learning_rate=0.0001)
model.build(loss_function='cross_entropy', optimizer=optimizer, metrics='accuracy')

In [10]:
%%time
model.train(x_train, y_train, epochs=10, batch_size=128, verbose=True)

Training on 60000 samples
Epoch 1/10
cross_entropy: 1.6821 - accuracy: 0.5391
Training on 60000 samples
Epoch 2/10
cross_entropy: 1.3679 - accuracy: 0.6484
Training on 60000 samples
Epoch 3/10
cross_entropy: 1.1780 - accuracy: 0.7266
Training on 60000 samples
Epoch 4/10
cross_entropy: 0.9567 - accuracy: 0.7734
Training on 60000 samples
Epoch 5/10
cross_entropy: 0.8732 - accuracy: 0.7422
Training on 60000 samples
Epoch 6/10
cross_entropy: 0.9316 - accuracy: 0.7266
Training on 60000 samples
Epoch 7/10
cross_entropy: 0.7509 - accuracy: 0.7422
Training on 60000 samples
Epoch 8/10
cross_entropy: 0.7198 - accuracy: 0.7578
Training on 60000 samples
Epoch 9/10
cross_entropy: 0.5410 - accuracy: 0.8125
Training on 60000 samples
Epoch 10/10
cross_entropy: 0.5575 - accuracy: 0.8281
CPU times: user 2min 55s, sys: 9.12 s, total: 3min 5s
Wall time: 20.7 s


In [11]:
model.evaluate(x_test, y_test)

'cross_entropy: 0.6207 - accuracy: 0.8025'